# Question 2

- This problem is aimed at recommanding products by finding products that are frequently browsered together 

In [3]:
from pyspark import SparkConf, SparkContext
import re
from itertools import combinations, permutations

In [4]:
SUPPORT = 100

In [5]:
conf = SparkConf()
sc = SparkContext(conf=conf)
lines = sc.textFile("q2/data/browsing.txt")

In [6]:
baskets = lines.map(lambda line: \
                    re.split(r'[^\w]+', line))\
              .map(lambda line: [v for v in line[0:-1]])

In [7]:
freq_items_1st = baskets.map(lambda line: [(v,1) for v in line])\
                        .flatMap(lambda x: x)\
                        .reduceByKey(lambda n1,n2: n1+n2)\
                        .filter(lambda x: x[1]>=SUPPORT)

keys_1st = freq_items_1st.keys()\
                         .collect()
dict_1st = dict(freq_items_1st.collect())

## 2d

In [8]:
freq_items_2nd = baskets.filter(lambda x: [v for v in x if v in keys_1st])\
                    .flatMap(lambda x: \
                             [(key, 1) for key in permutations(x, 2)])\
                    .reduceByKey(lambda n1,n2: n1+n2)\
                    .filter(lambda x: x[1] >= SUPPORT)

keys_2nd = freq_items_2nd.keys().collect()

dict_2nd = dict(freq_items_2nd.collect())

In [9]:
confidence_2nd = freq_items_2nd.map(lambda x: (x[0], x[1]/dict_1st[x[0][0]]))\
                               .sortBy(lambda x: (-x[1], x[0][0]))

In [10]:
confidence_2nd.take(5)

[(('DAI93865', 'FRO40251'), 1.0),
 (('GRO85051', 'FRO40251'), 0.999176276771005),
 (('GRO38636', 'FRO40251'), 0.9906542056074766),
 (('ELE12951', 'FRO40251'), 0.9905660377358491),
 (('DAI88079', 'FRO40251'), 0.9867256637168141)]

## 2e

In [11]:
def sorttuple(l):
    '''sort each tuple in l'''
    l1 = []
    for v in l:
        l1.append(tuple(sorted(v)))
    return l1

In [12]:
def expand(line):
    '''expand pairs in the line to single elements'''
    l = []
    for v in line:
        l.append(v[0])
        l.append(v[1])
    l = list(set(l))
    return l

In [13]:
def prune_3rd(line, keys):
    '''prune the triples which are not qualified'''
    
    l = []
    
    for v in line:
        p = 1
        for pair in combinations(v,2):
            if pair not in keys: 
                p = 0
                break
        if p == 1: l.append(v)
            
    return l

In [14]:
freq_1_2nd = expand(keys_2nd)

In [15]:
k2 = [tuple(sorted(v)) for v in list(set(sorttuple(keys_2nd)))]

In [16]:
freq_items_3rd = baskets.filter(lambda x: [v for v in x if v in freq_1_2nd])\
                        .map(lambda x: [tuple(sorted(v)) for v in combinations(x, 3)])\
                        .flatMap(lambda x: [(v,1) for v in x if (v[0], v[1]) in k2 and (v[0], v[2]) in k2 and (v[1], v[2]) in k2])\
                        .reduceByKey(lambda n1, n2: n1+n2)\
                        .filter(lambda x: x[1] >= SUPPORT)

In [17]:
confidence_3rd = freq_items_3rd.flatMap(lambda x: \
                                        [((x[0][0:2], x[0][2]), x[1]/dict_2nd[x[0][0:2]]),\
                                         ((x[0][1:3], x[0][0]), x[1]/dict_2nd[x[0][1:3]]),\
                                         (((x[0][0], x[0][2]), x[0][1]), x[1]/dict_2nd[(x[0][0], x[0][2])])])\
                               .sortBy(lambda x: (-x[1], x[0][0], x[0]))

In [18]:
confidence_3rd.take(5)

[((('DAI23334', 'ELE92920'), 'DAI62779'), 1.0),
 ((('DAI31081', 'GRO85051'), 'FRO40251'), 1.0),
 ((('DAI55911', 'GRO85051'), 'FRO40251'), 1.0),
 ((('DAI62779', 'DAI88079'), 'FRO40251'), 1.0),
 ((('DAI75645', 'GRO85051'), 'FRO40251'), 1.0)]